# Compare the results of various experiments involving 1 robot for 1 day

This notebook creates graphs that compare the results of various experimental runs that involved running 1 robot for 1 day. 

This comparison is described by an exp/run of type 1robot1day, starting with the prefix comp_xxx. The sub-experiments are listed in that exp/run file.

The assumption is that those experiments had been already run and their results had been stored in the results.pickle files in the corresponding data dirs. 

In [ ]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "WaterBerryFarms"
import pathlib
#from pprint import pprint
import gzip as compress
#import copy
import pickle
import matplotlib.pyplot as plt
import numpy as np

import wbf_figures

ImportError: cannot import name 'add_robot_path' from 'wbf_figures' (c:\Users\lboloni\Documents\Code\_Checkouts\WBF\WaterBerryFarms\notebooks\..\wbf_figures.py)

In [ ]:
experiment = "1robot1day"

# the lead run is the one that will provide the test data.
# also the directory of this run is where the graphs will be saved.

# leadrun = "comp_all"
leadrun = "comp_ad"

# now load them

leadexp = Config().get_experiment(experiment, leadrun)

runs = leadexp["tocompare"]

exps = []
allresults = [ ]
for run in runs:
    exp = Config().get_experiment(experiment, run)
    exps.append(exp)
    resultsfile = pathlib.Path(exp["data_dir"], "results.pickle")
    if not resultsfile.exists():
        print(f"Results file does not exist:\n{resultsfile}")
        print("Run the notebook Run-1Robot1Day with the same exp/run to create it.")
        raise Exception("Nothing to do.")
    with compress.open(resultsfile, "rb") as f:
        results = pickle.load(f)
    allresults.append(results)

In [ ]:
graphfilename = pathlib.Path(leadexp["data_dir"], "end-of-day.pdf")
wbf_figures.end_of_day_graphs(allresults[0], graphfilename = graphfilename, plot_uncertainty = False)

In [ ]:
fig, ax_scores = plt.subplots(1, figsize=(6,4))
for results in allresults:
    ax_scores.plot(results["scores"], label = f'{results["policy-code"].name[len("benchmarkpaper-"):]}+{results["estimator-code"].name}')
ax_scores.set_ylim(top=0)
ax_scores.set_xlabel("Time")
ax_scores.set_ylabel("Score")
ax_scores.set_title("Scores")
ax_scores.legend()
fig.tight_layout()

In [ ]:
horizontal = True
labels = ["food", "bar"]
metric="tylcv"
uncmap = "inferno" # was "grey", "viridis"

In [ ]:
if horizontal:
    fig, axes = plt.subplots(nrows=3, ncols=(1+len(labels)), figsize=(3*(1+len(labels)),  3*3))
    #b = [ a[i][0] for i in range(len(a))]
    firstax = [ axes[i][0] for i in range(len(axes))]
else:    
    fig, axes = plt.subplots(nrows=(1+len(labels)), ncols=3, figsize=(3*3,3*(1+len(labels))))
    firstax = axes[0]
# print(firstax)
firstax[0].axis('off')
firstax[2].axis('off')

# graph_env_im(allresults[labels[0]]["wbfe"], allresults[labels[0]]["estimator-code"], ax_env_tylcv=axes[0][1])
graph_env_im(allresults[0]["wbfe"], allresults[0]["estimator-code"], **{f"ax_env_{metric}":firstax[1]})
axes[1][0].set_title(f"Ground truth {metric}")
filename = f"gt-{metric}-"
for i, label in enumerate(labels):        
    results = allresults[i]
    filename += label + "-"
    if horizontal:
        axrow = [ axes[j][i+1] for j in range(len(axes))]
        #axrow = axes[:][i+1]
    else:    
        axrow = axes[i+1]
    # empty = np.ones_like(results["wbfe"].tylcv.value.T)
    empty = np.ones_like(vars(results["wbfe"])[metric].value.T)
    image_env = axrow[0].imshow(empty, vmin=0, vmax=1, origin="lower", cmap="gray")    
    wbf_figures.show_robot_path(results, axrow[0], draw_robot = False)
    # graph_env_im(results["wbfe"], results["estimator-code"], ax_im_tylcv=axrow[1])
    graph_env_im(results["wbfe"], results["estimator-code"], **{f"ax_im_{metric}" : axrow[1]})
    # graph_env_im(results["wbfe"], results["estimator-code"], ax_unc_tylcv = axrow[2], cmap=uncmap)
    graph_env_im(results["wbfe"], results["estimator-code"], cmap=uncmap, **{f"ax_unc_{metric}": axrow[2]})
    axrow[0].set_title(f"{label} path")
    axrow[1].set_title(f"{label} estimate")
    axrow[2].set_title(f"{label} uncertainty")
plt.tight_layout()
filename = filename[:-1] + ".pdf"
plt.savefig(pathlib.Path(results["results-basedir"], filename)) 